# Hiatuses

In [ ]:
import os
import sys
sys.path.append("..")
import numpy as np
import scipy as sp
import xarray as xr
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2
%aimport - numpy - matplotlib.pyplot

In [ ]:
from GMST import Hiatuses, GMST_GISTEMP
from paths import path_results, CESM_filename
from timeseries import IterateOutputCESM

# Trends in observations

In [ ]:
GISTEMP = GMST_GISTEMP()

In [ ]:
period = {'time':slice(1960,2017)}
lf = np.polyfit(GISTEMP.time.sel(period), GISTEMP.GMST.sel(period), 1)
GISTEMP_lin_trend = [lf[0]*10]*len(GISTEMP.time.sel(period))

In [ ]:
f, ax = plt.subplots(1,1, figsize=(8,5), sharex=True)
ax.tick_params(labelsize=14)
ax.axhline(0, c='k', lw=.5)
ax.plot(GISTEMP.time.sel(period), lf[1] + lf[0]*GISTEMP.time.sel(period), c='grey', ls='--', label='lin. fit 1960-2017')
ax.plot(GISTEMP.time, GISTEMP.GMST, label='GISTEMP')
ax.set_xlabel('time [year]', fontsize=16)
ax.set_ylabel('GMST anomaly [K]', fontsize=16)
ax.legend(fontsize=14)
f.tight_layout()
f.savefig(f'{path_results}/HIATUS/GISTEMP_anomaly')

f, ax = plt.subplots(1,1, figsize=(8,5), sharex=True)
ax.tick_params(labelsize=14)
ax.axhline(0, c='k', lw=.5)
ax.plot(GISTEMP.time.sel(period), GISTEMP_lin_trend, c='grey', ls='--', label='lin. fit 1960-2017')
ax.plot(GISTEMP.time, GISTEMP.trend_10*10, c='C1', ls=':' , label='10 yr lin. trend')
ax.plot(GISTEMP.time, GISTEMP.trend_15*10, c='C2', ls='--', label='15 yr lin. trend')
ax.plot(GISTEMP.time, GISTEMP.trend_30*10, c='C3', ls='-.', label='30 yr lin. trend')
ax.set_xlabel('time [year]', fontsize=16)
ax.set_ylabel('trend [K/decade]', fontsize=16)
ax.legend(fontsize=14)
f.tight_layout()
f.savefig(f'{path_results}/HIATUS/GISTEMP_trends')

f, ax = plt.subplots(1,1, figsize=(8,5), sharex=True)
ax.tick_params(labelsize=14)
ax.axhline(0, c='k', lw=.5)
ax.plot(GISTEMP.time.sel(period), GISTEMP.GMST - lf[1] - lf[0]*GISTEMP.time.sel(period), c='C0', ls='-', label='GISTEMP - lin. fit 1960-2017')
ax.set_xlabel('time [year]', fontsize=16)
ax.set_ylabel('corrected GMST anomaly [K]', fontsize=16)
ax.legend(fontsize=14)
f.tight_layout()
f.savefig(f'{path_results}/HIATUS/GISTEMP_anomaly')

f, ax = plt.subplots(1,1, figsize=(8,5), sharex=True)
ax.tick_params(labelsize=14)
ax.axhline(0, c='k', lw=.5)
ax.axhline(-.07, c='k', lw=.5, ls='--')
ax.axhline(-.17, c='k', lw=.5, ls='--')
ax.plot(GISTEMP.time.sel(period), GISTEMP.trend_10.sel(period)*10-GISTEMP_lin_trend, c='C1', ls=':' , label='10 yr lin. trend')
ax.plot(GISTEMP.time.sel(period), GISTEMP.trend_15.sel(period)*10-GISTEMP_lin_trend, c='C2', ls='--', label='15 yr lin. trend')
ax.plot(GISTEMP.time.sel(period), GISTEMP.trend_30.sel(period)*10-GISTEMP_lin_trend, c='C3', ls='-.', label='30 yr lin. trend')
ax.set_xlabel('time [year]', fontsize=16)
ax.set_ylabel('corrected trend [K/decade]', fontsize=16)
ax.legend(fontsize=14)
f.tight_layout()
# f.savefig(f'{path_results}/HIATUS/GISTEMP_trends')

# for i in range(2):
#     ax[i].spines['right'].set_visible(False)
#     ax[i].spines['top'].set_visible(False)

# f.align_ylabels()


# Identifying hiatuses in CESM

In [ ]:
gmst_ctrl = xr.open_dataset(f'{path_results}/GMST/GMST_ctrl.nc')
gmst_rcp  = xr.open_dataset(f'{path_results}/GMST/GMST_rcp.nc' )
gmst_lpd  = xr.open_dataset(f'{path_results}/GMST/GMST_lpd.nc' )
gmst_lpi  = xr.open_dataset(f'{path_results}/GMST/GMST_lpi.nc' )

In [ ]:
lfit_ctrl = gmst_ctrl.lin_fit .attrs['lin_fit_params']
qfit_ctrl = gmst_ctrl.quad_fit.attrs['quad_fit_params']
lfit_rcp  = gmst_rcp .lin_fit .attrs['lin_fit_params']
qfit_rcp  = gmst_rcp .quad_fit.attrs['quad_fit_params']
lfit_lpd  = gmst_lpd .lin_fit .attrs['lin_fit_params']
qfit_lpd  = gmst_lpd .quad_fit.attrs['quad_fit_params']
lfit_lpi  = gmst_lpi .lin_fit .attrs['lin_fit_params']
qfit_lpi  = gmst_lpi .quad_fit.attrs['quad_fit_params']

In [ ]:
lfit_ctrl_trend = np.empty((len(gmst_ctrl.time)))
qfit_ctrl_trend = np.empty((len(gmst_ctrl.time)))
for i, t in enumerate(gmst_ctrl.time.values):  
    lfit_ctrl_trend[i] = lfit_ctrl[0]    
    qfit_ctrl_trend[i] = 2*qfit_ctrl[0]*i + qfit_ctrl[1]
    
lfit_rcp_trend  = np.empty((len(gmst_rcp.time)))
qfit_rcp_trend  = np.empty((len(gmst_rcp.time)))
for i, t in enumerate(gmst_rcp.time.values):  
    lfit_rcp_trend[i] = lfit_rcp[0]    
    qfit_rcp_trend[i] = 2*qfit_rcp[0]*i + qfit_rcp[1]

In [ ]:
f, ax = plt.subplots(2,2, figsize=(12,7.5), sharex='col')

for i in range(2):
    ax[1,i].axhline(0, c='k', lw=.5)
    ax[1,i].set_xlabel('time [years]', fontsize=16)
    for j in range(2):
        ax[i,j].tick_params(labelsize=14)
        ax[i,j].spines['right'].set_visible(False)
        ax[i,j].spines['top'].set_visible(False)
    ax[0,i].set_ylabel('GMST [$^\circ$C]', fontsize=16)
    ax[1,i].set_ylabel('lin. trend [$^\circ$C/dec]', fontsize=16)

ax[0,0].plot(gmst_ctrl.time/365, gmst_ctrl.lin_fit , c='grey', ls='-' , label='lin. fit')
ax[0,0].plot(gmst_ctrl.time/365, gmst_ctrl.quad_fit, c='grey', ls='--', label='quad. fit')
ax[0,0].plot(gmst_ctrl.time/365, gmst_ctrl.GMST, lw=2, c='C0', label='')
ax[0,0].text(0.02, 0.9, 'CTRL', transform=ax[0,0].transAxes, fontsize=16)
ax[0,0].legend(loc=4, fontsize=14)

ax[1,0].plot(gmst_ctrl.time/365, lfit_ctrl_trend*10, c='grey', ls='-' )
ax[1,0].plot(gmst_ctrl.time/365, qfit_ctrl_trend*10, c='grey', ls='--')
ax[1,0].plot(gmst_ctrl.time/365, qfit_ctrl_trend*10-.07, c='g', ls='--')
ax[1,0].plot(gmst_ctrl.time/365, qfit_ctrl_trend*10-.17, c='g', ls='--')
ax[1,0].plot(gmst_ctrl.time/365, gmst_ctrl.trend_10*10, lw=1, c='C0', ls='--' , label='10 yrs')
ax[1,0].plot(gmst_ctrl.time/365, gmst_ctrl.trend_15*10, lw=2, c='C0', ls='-' , label='15 yrs')
ax[1,0].plot(gmst_ctrl.time/365, gmst_ctrl.trend_30*10, lw=1, c='C0', ls=':', label='30 yrs')
ax[1,0].legend(fontsize=14, ncol=3, frameon=False)

ax[0,1].plot(gmst_rcp.time/365, gmst_rcp.lin_fit , c='grey', ls='-' )
ax[0,1].plot(gmst_rcp.time/365, gmst_rcp.quad_fit, c='grey', ls='--')
ax[0,1].plot(gmst_rcp.time/365, gmst_rcp.GMST, lw=2, c='C1')
ax[0,1].text(0.02, 0.9, 'RCP', transform=ax[0,1].transAxes, fontsize=16)

ax[1,1].plot(gmst_rcp.time/365, lfit_rcp_trend*10, c='grey', ls='-' )
ax[1,1].plot(gmst_rcp.time/365, qfit_rcp_trend*10, c='grey', ls='--')
ax[1,1].plot(gmst_rcp.time/365, qfit_rcp_trend*10-.07, c='g', ls='--')
ax[1,1].plot(gmst_rcp.time/365, qfit_rcp_trend*10-.17, c='g', ls='--')
ax[1,1].plot(gmst_rcp.time/365, gmst_rcp.trend_10 *10, lw=1, c='C1', ls='--' , label='10 yrs')
ax[1,1].plot(gmst_rcp.time/365, gmst_rcp.trend_15 *10, lw=2, c='C1', ls='-' , label='15 yrs')
ax[1,1].plot(gmst_rcp.time/365, gmst_rcp.trend_30 *10, lw=1, c='C1', ls=':', label='30 yrs')
ax[1,1].legend(fontsize=14, ncol=3, frameon=False)

f.tight_layout()
f.align_ylabels()
f.savefig(f'{path_results}/HIATUS/HIATUS_GMST_rolling_trends')

In [ ]:
f, ax = plt.subplots(2,1, figsize=(12,7.5), sharex='col')

for i in range(2):
    ax[i].axhline(0, c='k', lw=.5)
    ax[i].tick_params(labelsize=14)
    ax[i].spines['right'].set_visible(False)
    ax[i].spines['top'].set_visible(False)
    ax[i].axvline(200, c='g', lw=.5)
    ax[i].axvline(300, c='g', lw=.5)
ax[0].set_ylabel('GMST anom. [$^\circ$C]', fontsize=16)
ax[1].set_xlabel('time [years]', fontsize=16)
ax[1].set_ylabel('lin. trend anom. [$^\circ$C/dec]', fontsize=16)
ax[1].set_xlim((95,390))
ax[1].set_xticks(np.arange(100,400,20))


ax[0].plot(gmst_ctrl.time/365    , gmst_ctrl.GMST-gmst_ctrl.quad_fit, lw=1, c='C0')
ax[0].plot(gmst_rcp.time/365-1700, gmst_rcp .GMST-gmst_rcp .quad_fit, lw=1, c='C1', label='yearly data')
ax[0].plot(gmst_ctrl.time/365    , (gmst_ctrl.GMST-gmst_ctrl.quad_fit).rolling(time=10, center=True).mean(), lw=2, c='C0')
ax[0].plot(gmst_rcp.time/365-1700, (gmst_rcp .GMST-gmst_rcp .quad_fit).rolling(time=10, center=True).mean(), lw=2, c='C1', label='10 year running mean')
ax[0].text(305, 0.2, 'RCP',  fontsize=16)
ax[0].text(105, 0.2, 'CTRL', fontsize=16)
ax[0].legend(fontsize=14 ,ncol=2, frameon=False)

ax[1].axhline(-.07, c='k', lw=.5, ls=':')
ax[1].axhline(-.17, c='k', lw=.5, ls=':')
ax[1].plot(gmst_ctrl.time/365, (gmst_ctrl.trend_10-qfit_ctrl_trend)*10, lw=1, c='C0', ls='--')
ax[1].plot(gmst_ctrl.time/365, (gmst_ctrl.trend_15-qfit_ctrl_trend)*10, lw=1, c='C0', ls='-' )
L1, = ax[1].plot(gmst_rcp.time/365-1700, (gmst_rcp.trend_10-qfit_rcp_trend) *10, lw=1, c='C1', ls='--', label='10 yrs')
L2, = ax[1].plot(gmst_rcp.time/365-1700, (gmst_rcp.trend_15-qfit_rcp_trend) *10, lw=1, c='C1', ls='-' , label='15 yrs')
ax[1].legend(handles=[L1,L2], fontsize=14, ncol=6, frameon=False, loc=4)

hiatuses = [item for sublist in [Hiatuses('ctrl'), Hiatuses('rcp',-1700)] for item in sublist]
for i in range(2):
    for hiat in hiatuses:
        ax[i].axvspan(hiat[0],hiat[1], alpha=.2)
    
f.tight_layout()
f.align_ylabels()
f.savefig(f'{path_results}/HIATUS/HIATUS_GMST_rolling_trends_anom')

In [ ]:
# low res pd run
f, ax = plt.subplots(2,1, figsize=(12,8), sharex=True)
for i in range(2):
    ax[i].tick_params(labelsize=14)
    ax[i].spines['right'].set_visible(False)
    ax[i].spines['top'].set_visible(False)

ax[0].set_ylabel('GMST [$^\circ$C]', fontsize=16)
ax[0].plot(gmst_lpd.time/365, gmst_lpd.GMST)
ax[0].plot(gmst_lpd.time/365, gmst_lpd.GMST.rolling(time=30, center=True).mean())
ax[0].plot(gmst_lpd.time/365, gmst_lpd.lin_fit, c='grey')
ax[0].text(.98,.02, f'linear trend +{lfit_lpd[0]*1e2:4.2f} K/century', 
           ha='right', transform=ax[0].transAxes, fontsize=14, color='grey')

ax[1].set_ylabel('lin. trends [$^\circ$C/decade]', fontsize=16)
ax[1].axhline(0, c='k', lw=.5)
ax[1].axhline(-.07, c='g', lw=.5)
ax[1].axhline(-.17, c='g', lw=.5)
ax[1].plot(gmst_lpd.time/365, (gmst_lpd.trend_10-lfit_lpd[0])*10, label='10 yrs')
ax[1].plot(gmst_lpd.time/365, (gmst_lpd.trend_15-lfit_lpd[0])*10, label='15 yrs')
ax[1].plot(gmst_lpd.time/365, (gmst_lpd.trend_30-lfit_lpd[0])*10, label='30 yrs')
ax[1].legend(fontsize=14, frameon=False)

ax[1].set_xlabel('time [years]', fontsize=16)
f.align_ylabels()
plt.tight_layout()
f.savefig(f'{path_results}/HIATUS/HIATUS_lpd')

In [ ]:
# low res pd run
f, ax = plt.subplots(2,1, figsize=(12,8), sharex=True)
for i in range(2):
    ax[i].tick_params(labelsize=14)

ax[0].set_ylabel('GMST [$^\circ$C]', fontsize=16)
ax[0].plot(gmst_lpi.time/365, gmst_lpi.GMST)
ax[0].plot(gmst_lpi.time/365, gmst_lpi.GMST.rolling(time=30, center=True).mean())
ax[0].plot(gmst_lpi.time/365, gmst_lpi.lin_fit, c='grey')
ax[0].text(.98,.02, f'linear trend +{lfit_lpi[0]*1e2:4.2f} K/century', 
           ha='right', transform=ax[0].transAxes, fontsize=14, color='grey')

ax[1].set_ylabel('lin. trends [$^\circ$C/decade]', fontsize=16)
ax[1].axhline(0, c='k', lw=.5)
ax[1].axhline(-.07, c='g', lw=.5)
ax[1].axhline(-.17, c='g', lw=.5)
ax[1].plot(gmst_lpi.time/365, (gmst_lpi.trend_10-lfit_lpd[0])*10, label='10 yrs')
ax[1].plot(gmst_lpi.time/365, (gmst_lpi.trend_15-lfit_lpd[0])*10, label='15 yrs')
ax[1].plot(gmst_lpi.time/365, (gmst_lpi.trend_30-lfit_lpd[0])*10, label='30 yrs')
ax[1].legend(fontsize=14, frameon=False)

ax[1].set_xlabel('time [years]', fontsize=16)
f.align_ylabels()
plt.tight_layout()
f.savefig(f'{path_results}/HIATUS/HIATUS_lpi')

In [ ]:
gmst_ctrl

In [ ]:
from GMST import rolling_lin_trends

In [ ]:
# distribution of trends
gmsts  = [gmst_ctrl, gmst_rcp, gmst_lpd, gmst_lpi]
labels = ['CTRL', 'RCP', 'LPD', 'LPI']
x = np.arange(-.55,.55,0.001)

f, ax = plt.subplots(2, 2, figsize=(12,8), sharey='row', sharex='col')
for i in range (4):
    label= labels[i]
    gmst = gmsts[i].copy()
    density = sp.stats.gaussian_kde(gmst.trend_10.dropna(dim='time')*10-gmst.trend_10.dropna(dim='time').mean()*10)
    ax[0,0].plot(x, density(x), label=label)
    density = sp.stats.gaussian_kde(gmst.trend_15.dropna(dim='time')*10-gmst.trend_15.dropna(dim='time').mean()*10)
    ax[0,1].plot(x, density(x), label=label)
#     if i==1: ax[0,2].plot((gmst.GMST-gmst.lin_fit)*10)
    
    gmst['GMST'] = gmst.GMST-gmst.quad_fit
    gmst = rolling_lin_trends(gmst,len(gmst.GMST), gmst.time)
    
    density = sp.stats.gaussian_kde(gmst.trend_10.dropna(dim='time')*10-gmst.trend_10.dropna(dim='time').mean()*10)
    ax[1,0].plot(x, density(x), label=label)
    density = sp.stats.gaussian_kde(gmst.trend_15.dropna(dim='time')*10-gmst.trend_15.dropna(dim='time').mean()*10)
    ax[1,1].plot(x, density(x), label=label)
#     if i==1: ax[1,2].plot((gmst.GMST)*10)
    
ax[1,0].set_xlabel('10 year trends [K/decade]', fontsize=16)
ax[1,1].set_xlabel('15 year trends [K/decade]', fontsize=16)

for i in range(2):
    ax[i,0].set_ylabel('density', fontsize=16)
    ax[i,0].axvline(-.17, c='k', lw=.5, ls='--')
    ax[i,1].axvline(-.07, c='k', lw=.5, ls='--')
    ax[0,i].text(.05, .85, 'linearly\ndetrended'     , fontsize=14, transform=ax[0,i].transAxes, color='grey')
    ax[1,i].text(.05, .85, 'quadratically\ndetrended', fontsize=14, transform=ax[1,i].transAxes, color='grey')
    for j in range(2):
        ax[i,j].axvline(0, c='k', lw=.5)
        ax[i,j].tick_params(labelsize=14)
        ax[i,j].axhline(0, c='k', lw=.5)
        ax[i,j].legend(fontsize=14, frameon=False)
f.align_ylabels()
plt.tight_layout()
plt.savefig(f'{path_results}/HIATUS/HIATUS_symmetry')

In [ ]:
gmst.GMST.plot()

In [ ]:
Hiatuses('rcp')-1700

In [ ]:
hiat = [item for sublist in [Hiatuses('ctrl'), Hiatuses('rcp',-1700)] for item in sublist]

In [ ]:
hiat

In [ ]:
gmst_trend_anom_ctrl = xr.merge([gmst_ctrl.trend_10-qfit_ctrl_trend,
                                 gmst_ctrl.trend_15-qfit_ctrl_trend])
gmst_trend_anom_rcp  = xr.merge([gmst_rcp .trend_10-qfit_rcp_trend,
                                 gmst_rcp .trend_15-qfit_rcp_trend])
gmst_trend_anom_ctrl.to_netcdf(f'{path_results}/GMST/GMST_trend_anom_ctrl.nc')
gmst_trend_anom_rcp .to_netcdf(f'{path_results}/GMST/GMST_trend_anom_rcp.nc' )

In [ ]:
# plt.avhline(0, c='k')
f,ax = plt.subplots(1,2, figsize=(12,5))
for i in range(2):
    ax[i].tick_params(labelsize=14)
ax[0].hist(gmst_trend_anom_ctrl.trend_10, bins=np.arange(-.035,.04,.005), color='C0', alpha=.4, label='CTRL 10 yr')
ax[0].hist(gmst_trend_anom_ctrl.trend_15, bins=np.arange(-.035,.04,.005), color='C1', alpha=.4, label='CTRL 15 yr')
ax[1].hist(gmst_trend_anom_rcp .trend_10, bins=np.arange(-.035,.04,.005), color='C0', alpha=.4, label='RCP 10 yr')
ax[1].hist(gmst_trend_anom_rcp .trend_15, bins=np.arange(-.035,.04,.005), color='C1', alpha=.4, label='RCP 15 yr')
for i in range(2):
    ax[i].legend()
    plt.tight_layout()


which thresholds to choose (from Hedemann 2016)?
- 0.17: mismatch between CMIP5 ensemble mean
- 0.04-0.07: deviation from long term trend

In [ ]:
gmst_trend_anom_ctrl